<a href="https://colab.research.google.com/github/HaswanthVIT/BakeScape/blob/main/BakeScape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install diffusers transformers accelerate xformers
!pip install pandas opencv-python Pillow scikit-image matplotlib
!pip install gradio wandb imagehash


Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 9.0 MB/s eta 0:00:00


Create folders

In [ ]:
# Create clean folder structure
!mkdir -p bakescape/{src,configs}
!mkdir -p data/{raw,processed}
!mkdir -p outputs/{gen,logs,ckpts}
!touch bakescape/__init__.py


Extract dataset info

In [ ]:
!unzip -q cakes-dataset.zip -d data/raw/


unzip:  cannot find or open cakes-dataset.zip, cakes-dataset.zip.zip or cakes-dataset.zip.ZIP.


In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


!pip install kaggle


In [3]:
!pip install kaggle


In [4]:
!kaggle datasets download -d ishikanaik/cakes-dataset -p data/raw/


Dataset URL: https://www.kaggle.com/datasets/ishikanaik/cakes-dataset
License(s): unknown
 59% 125M/213M [00:00<00:00, 1.31GB/s]
100% 213M/213M [00:00<00:00, 860MB/s] 


In [5]:
!unzip -q data/raw/cakes-dataset.zip -d data/raw/


In [16]:
!find data/raw/cakes-dataset/ -type f -name "*.jpg" -exec cp {} data/raw/ \;


In [15]:
!ls -R data/raw | head -50


data/raw:
cakes-dataset
cakes-dataset.zip

data/raw/cakes-dataset:
test
train
validation

data/raw/cakes-dataset/test:
cheesecake
chocolate
pie
red_velvet

data/raw/cakes-dataset/test/cheesecake:
3-Ingredient Souffle Cheesecake _Sweet... (1).jpg
3-Ingredient Souffle Cheesecake _Sweet... (2).jpg
3-Ingredient Souffle Cheesecake _Sweet... (3).jpg
3-Ingredient Souffle Cheesecake _Sweet....jpg
3-Step Mini Cheesecakes.jpg
5-Minute Microwave Cheesecake Recipe by....jpg
6 Inch Cheesecake Recipe ┬╗ Hummingbird High.jpg
No-Bake Cheesecake _ 8 easy homemade....jpg
No-Bake Cheesecake Cups Recipe....jpg
No-Bake Cheesecake - Live Well Bake Often.jpg
No-Bake Cheesecake Recipe _ Martha Stewart.jpg
No-Bake Cheesecake Recipe - Sugar Spun Run.jpg
No-Bake Cheesecake_ so fluffy smooth....jpg
No-Bake Cheesecake _ - Tastes Better... (1).jpg
No-Bake Cheesecake _ - Tastes Better... (2).jpg
No-Bake Cheesecake _ - Tastes Better... (3).jpg
No-Bake Cheesecake _ - Tastes Better....jpg
No-Bake Cherry Cheesecake {9x1

In [17]:
!ls data/raw | wc -l


2498


Create Index and EDA

In [7]:
!pip install imagehash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 20.5 MB/s eta 0:00:00


In [18]:
import os
import cv2
import pandas as pd
import imagehash
from PIL import Image

raw_dir = "data/raw/"
files = [f for f in os.listdir(raw_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))]

records = []
for f in files:
    path = os.path.join(raw_dir, f)
    try:
        img = cv2.imread(path)
        h, w, c = img.shape
        # perceptual hash for duplicate check
        phash = str(imagehash.phash(Image.open(path)))
        records.append([f, w, h, c, phash])
    except:
        print("Corrupted:", f)

df = pd.DataFrame(records, columns=["filename","width","height","channels","phash"])
df.to_csv("data_index.csv", index=False)
df.head()


,filename,width,height,channels,phash
0,Philadelphia - Recipe - Fresh Berry....jpg,1000,1000,3,f184ce59b0669bca
1,Cheesecake Recipe - Preppy Kitchen.jpg,1024,537,3,fab5a1da8f2498a4
2,ia_100001012.jpg,310,163,3,bccdc1c48f9e1d18
3,The Cheesecake Factory Frozen Original....jpg,488,488,3,f42d49f28834bccd
4,The Best Vegan Cheesecake - Nora Cooks (1).jpg,667,1000,3,c3bf60823f49f087


In [19]:
import pandas as pd

df_index = pd.read_csv("data_index.csv")
display(df_index)

,filename,width,height,channels,phash
0,Philadelphia - Recipe - Fresh Berry....jpg,1000,1000,3,f184ce59b0669bca
1,Cheesecake Recipe - Preppy Kitchen.jpg,1024,537,3,fab5a1da8f2498a4
2,ia_100001012.jpg,310,163,3,bccdc1c48f9e1d18
3,The Cheesecake Factory Frozen Original....jpg,488,488,3,f42d49f28834bccd
4,The Best Vegan Cheesecake - Nora Cooks (1).jpg,667,1000,3,c3bf60823f49f087
...,...,...,...,...,...
2491,ia_100000870.jpg,225,225,3,afe086ec9c35b886
2492,ia_300000950.jpg,201,251,3,82a70397df82daa3
2493,ia_100000700.jpg,183,275,3,ff12923f3d524036
2494,ia_500000918.jpg,275,183,3,ce663931c3d863c6


Duplicate Check

In [20]:
dupes = df[df.duplicated("phash", keep=False)].sort_values("phash")
dupes


,filename,width,height,channels,phash
215,ia_300000687.jpg,225,225,3,80363f36294d75c7
873,ia_100000005407.jpg,700,700,3,80363f36294d75c7
1320,ia_300000702.jpg,275,183,3,81bfdbf209583c2c
859,ia_100000005402.jpg,1134,757,3,81bfdbf209583c2c
1745,ia_100000005352.jpg,2560,1439,3,83423e99e5eeec41
...,...,...,...,...,...
2290,ia_300000415.jpg,259,194,3,fec0954d1274db19
1402,ia_300000406.jpg,225,225,3,ff749160102f70af
545,ia_100000004959.jpg,1200,1200,3,ff749160102f70af
505,ia_300000465.jpg,190,266,3,ff82a1c0d44ec32f


from matplotlib import pyplot as plt
_df_2['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['width'].plot(kind='hist', bins=20, title='width')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_4['height'].plot(kind='hist', bins=20, title='height')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_5.plot(kind='scatter', x='index', y='width', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['channels']
  ys = series['height']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_9.sort_values('channels', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('channels')
_ = plt.ylabel('height')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['channels']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'channels'}, axis=1)
              .sort_values('channels', ascending=True))
  xs = counted['channels']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_10.sort_values('channels', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('channels')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_11['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_12['width'].plot(kind='line', figsize=(8, 4), title='width')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['height'].plot(kind='line', figsize=(8, 4), title='height')
plt.gca().spines[['top', 'right']].set_visible(False)

Remove Duplicates

In [22]:
unique_df = df.drop_duplicates(subset="phash", keep="first")
print("After removing duplicates:", len(unique_df))

# Move only unique images into a new folder
import shutil

os.makedirs("data/unique", exist_ok=True)
for f in unique_df['filename']:
    src = os.path.join(raw_dir, f)
    dst = os.path.join("data/unique", f)
    shutil.copy(src, dst)


After removing duplicates: 2424


In [25]:
!ls data/unique | wc -l


2424


Summary Report

In [26]:
print("Cleaned dataset report")
print("======================")
print("Total images (after deduplication):", len(unique_df))
print("Unique perceptual hashes:", unique_df['phash'].nunique())
print("\nImage size distribution:")
print(unique_df[['width','height']].describe())


Cleaned dataset report
Total images (after deduplication): 2424
Unique perceptual hashes: 2424

Image size distribution:
             width       height
count  2424.000000  2424.000000
mean    467.242162   453.908828
std     566.017262   540.852522
min     100.000000   100.000000
25%     201.000000   194.000000
50%     259.000000   243.000000
75%     500.000000   410.250000
max    6625.000000  5617.000000


In [27]:
unique_df.sample(5)


,filename,width,height,channels,phash
1563,ia_100000783.jpg,225,225,3,aaa49578c57ba2b1
33,ia_100000922.jpg,183,275,3,a88ac70db4e11bcf
1385,ia_500000510.jpg,183,275,3,b9e0cb86f98a7074
1540,Perfect Cheesecake - Broma Bakery.jpg,1600,2400,3,bf9cc861e046c3f8
2119,ia_100000686.jpg,225,225,3,be1e892b74d2834d
